In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
from PIL import Image

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision as tv
from torchvision.transforms import v2
from torchvision.transforms.v2.functional import to_pil_image

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
from dataset import load_all
from arch import RoadSegmentationModel

In [4]:
X_train, Y_train = load_all()

In [14]:
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [6]:
model = RoadSegmentationModel(
    in_channels=3, out_channels=1,
    num_filters=16, dropout=0.07
)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCEWithLogitsLoss()

In [13]:
lossi = []

In [ ]:
%%time

model.train() 

for epoch in range(1):
    epoch_loss = 0.0
    b = 0
    for images, masks in train_loader:
        optimizer.zero_grad()
        
        logits = model(images)
        loss = loss_fn(logits, masks)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

        b += 1
        print(f"{b}:", loss.item())

    epoch_loss /= len(train_loader)
    lossi.append(epoch_loss)
    
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss}")

1: 0.37373125553131104
2: 0.34911295771598816
3: 0.3565100431442261


In [ ]:
# torch.save(model.state_dict(), 'checkpoints/unet_001.chk')

In [19]:
# checkpoint = torch.load('checkpoints/unet_001.chk', weights_only=True)
# model.load_state_dict(checkpoint)

<All keys matched successfully>